In [1]:
import psycopg2
import sys
import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import time , datetime , date

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as off

### Funciones de utilidad

In [2]:
pickle_storage = './dataframes/'
pickle_extension = '.pkl'

def get_pickle(dbname,name):
    filename = pickle_storage + dbname + '_'+ name + pickle_extension
    return pd.read_pickle(filename)

def put_pickle(dbname,name, frame):
    filename = pickle_storage  + dbname + '_'+ name + pickle_extension
    frame.to_pickle(filename)

### Cargamos el dataframe Joineado

In [3]:
df_joined = get_pickle('joined_1_second','btcusd_xrpbtc_xrpusd')

In [4]:
df_joined.head()

,btcusd_initial_ask_price,btcusd_initial_bid_price,btcusd_final_ask_price,btcusd_final_bid_price,xrpbtc_initial_ask_price,xrpbtc_initial_bid_price,xrpbtc_final_ask_price,xrpbtc_final_bid_price,xrpusd_initial_ask_price,xrpusd_initial_bid_price,xrpusd_final_ask_price,xrpusd_final_bid_price
start,,,,,,,,,,,,
2018-01-18 09:11:02,11712.0,11711.0,11712.0,11711.0,0.000132,0.000131,0.000132,0.000131,1.5338,1.5324,1.5338,1.5324
2018-01-18 09:11:03,11712.0,11711.0,11712.0,11709.0,0.000132,0.000131,0.000132,0.000131,1.5338,1.5324,1.5338,1.5324
2018-01-18 09:11:04,11712.0,11706.0,11709.0,11702.0,0.000132,0.000131,0.000132,0.000131,1.5338,1.5324,1.5337,1.5324
2018-01-18 09:11:05,11709.0,11699.0,11708.0,11699.0,0.000132,0.000131,0.000131,0.000131,1.5337,1.5302,1.5300,1.5296
2018-01-18 09:11:06,11708.0,11699.0,11707.0,11703.0,0.000131,0.000131,0.000131,0.000131,1.5300,1.5296,1.5300,1.5296


### Aplicamos el calculo de evaluacion de rentabilidad

#### Formula general: Qusd_final = Qusd( Qxrp( Qbitcoins(Qusd_inicial)))

Con
<li>Qbitcoins (Qusd_inicial) = (Qusd - Qusd * Cmm) / btcusd_BID</li>
<li>Qxrp (Qbitcoins) = (Qbitcoins - Qbitcoins * Cmt) / xrpbtc_ASK</li>
<li>Qusd (Qxrp) = (Qxrp - Qxrp * Cmt) * xrpusd_BID</li>

Para valores ctes de
<li>Qusd_inicial = 1000 ;</li> <li>Cmm = 0.001% ;</li> <li>Cmt = 0.002%</li>

Hacemos:
<li>Qusd_final = Qusd( Qxrp( Qbitcoins(Qusd_inicial)))</li>

Resultado:
<li>Si Qusd_final es mayor que Qusd_inicial -> arbitraje favorable</li>
<li>Si Qusd_final es menor o igual que Qusd_inicial, entonces arbitraje desfavorable</li>


#### Medir: 
<li>cantidad de arbitrjes favorables</li>
<li>ventana de tiempo que duran los mismos</li>
